In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load your cleaned dataset (without 'date' column)
df = pd.read_csv("data.csv")  # Replace with your CSV filename

# Features and target
X = df[['temperature', 'humidity']].values
y = df['label'].values  # Make sure you have a column 'label' as 0/1 for anomaly

# Normalize data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split into train, val, test sets (70% train, 15% val, 15% test)
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, random_state=42)

In [2]:
# Build a small model compatible with TinyML (for Arduino Uno, <32KB flash)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=8,
    validation_data=(X_val, y_val)
)


Epoch 1/30
44/44 [==============================] - 1s 4ms/step - loss: 0.6161 - accuracy: 0.8281 - val_loss: 0.5786 - val_accuracy: 0.8421
Epoch 2/30
44/44 [==============================] - 0s 1ms/step - loss: 0.5655 - accuracy: 0.8080 - val_loss: 0.5330 - val_accuracy: 0.8289
Epoch 3/30
44/44 [==============================] - 0s 989us/step - loss: 0.5271 - accuracy: 0.7994 - val_loss: 0.4934 - val_accuracy: 0.8158
Epoch 4/30
44/44 [==============================] - 0s 976us/step - loss: 0.4985 - accuracy: 0.7937 - val_loss: 0.4650 - val_accuracy: 0.8158
Epoch 5/30
44/44 [==============================] - 0s 971us/step - loss: 0.4817 - accuracy: 0.7937 - val_loss: 0.4488 - val_accuracy: 0.8158
Epoch 6/30
44/44 [==============================] - 0s 974us/step - loss: 0.4713 - accuracy: 0.7937 - val_loss: 0.4405 - val_accuracy: 0.8158
Epoch 7/30
44/44 [==============================] - 0s 970us/step - loss: 0.4639 - accuracy: 0.7937 - val_loss: 0.4352 - val_accuracy: 0.8158
Epoch 8/30

In [3]:
# Evaluate
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"\n✅ Validation Accuracy: {val_acc * 100:.2f}%")

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")

3/3 [==============================] - 0s 1ms/step - loss: 0.3621 - accuracy: 0.8421

✅ Validation Accuracy: 84.21%
3/3 [==============================] - 0s 1ms/step - loss: 0.3705 - accuracy: 0.8800
✅ Test Accuracy: 88.00%


In [5]:
# Save the model
model.save("base_model.h5")


In [6]:
import tensorflow as tf

# Convert model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optimize for Microcontrollers
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save modelP
with open("anomaly_model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpilr0dwyy/assets


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpilr0dwyy/assets
2025-06-11 22:25:48.250659: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-06-11 22:25:48.250675: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-06-11 22:25:48.251010: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpilr0dwyy
2025-06-11 22:25:48.251777: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-06-11 22:25:48.251784: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpilr0dwyy
2025-06-11 22:25:48.253624: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-06-11 22:25:48.254637: I tensorflow/cc/saved_model/load

In [3]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="anomaly_model.tflite")
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Run inference on test data
y_pred = []
for sample in X_test:  # Make sure X_test is scaled
    input_data = np.array([sample], dtype=np.float32)  # Batch of 1

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    output = interpreter.get_tensor(output_details[0]['index'])[0][0]
    y_pred.append(1 if output > 0.5 else 0)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ TFLite Model Test Accuracy: {accuracy * 100:.2f}%")


✅ TFLite Model Test Accuracy: 88.00%


In [14]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.lite.python.interpreter import Interpreter

# Paths
h5_path = "base_model.h5"
tflite_path = "anomaly_model.tflite"
quant_path = "quant.tflite"

# Get and print model sizes
def print_model_sizes():
    h5_size_kb = os.path.getsize(h5_path) / 1024
    tflite_size_kb = os.path.getsize(tflite_path) / 1024
    quant_size_kb = os.path.getsize(quant_path) / 1024

    print(f"\n📦 H5 Model Size: {h5_size_kb:.2f} KB")
    print(f"📦 TFLite Model Size: {tflite_size_kb:.2f} KB")
    print(f"📦 Quant Model Size: {quant_size_kb:.2f} KB")

    if quant_size_kb < 32:
        print("✅ Quantized model is suitable for Arduino Uno (size < 32KB)")
    else:
        print("❌ Quantized model is NOT suitable for Arduino Uno (size >= 32KB)")
    return quant_path

# Load interpreter & estimate runtime and memory
def evaluate_model(tflite_model_path):
    interpreter = Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_shape = input_details[0]['shape']

    # Create dummy input for test
    input_data = np.random.rand(*input_shape).astype(np.float32)

    # Warm-up
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # Run inference multiple times
    times = []
    for _ in range(100):
        start = time.time()
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        end = time.time()
        times.append(end - start)

    avg_time_ms = np.mean(times) * 1000
    max_mem_kb = interpreter._get_tensor_details()[-1]['tensor'].nbytes / 1024

    print(f"\n🧠 Estimated Peak Memory Use (RAM): ~{max_mem_kb:.2f} KB (for output tensor only)")
    print(f"⚡ Avg Inference Time (on this system): {avg_time_ms:.2f} ms")

# Run everything
quant_path_used = print_model_sizes()
evaluate_model(quant_path_used)



📦 H5 Model Size: 32.45 KB
📦 TFLite Model Size: 2.27 KB
📦 Quant Model Size: 2.87 KB
✅ Quantized model is suitable for Arduino Uno (size < 32KB)


ValueError: Cannot set tensor: Got value of type FLOAT32 but expected type UINT8 for input 0, name: serving_default_dense_input:0 